In [1]:
import cv2
import re
import pytesseract
import os
import numpy as np
from matplotlib import pyplot as plt
from pytesseract import Output


# Métoddos

In [2]:
def show_img(img):
    plt.figure(figsize = (25,15))
    plt.imshow(img)
    plt.title('Cupom')
    plt.plot(8,8)
    plt.show()

In [3]:
def busca_dados(arquivo):
    
    nome_arquivo, extensao_arquivo = os.path.splitext(arquivo)
    
    print(f'Extensão: {extensao_arquivo}')
    if (extensao_arquivo != '.png' and extensao_arquivo != '.jpg'):
        return None
    
    print(f'Processando: {nome_arquivo}')
    
    data_compra = ''
    cnpj = ''
    total = ''

    img_cupom = cv2.imread(arquivo)
    img_scale = cv2.resize(img_cupom, (0, 0), fx=2, fy=2)
    img_gray = cv2.cvtColor(img_scale, cv2.COLOR_BGR2GRAY)


    custom_config = r'--oem 3 --psm 6 -l por'
    date_pattern = '^(0?[1-9]|[12][0-9]|3[01])[\/\-](0?[1-9]|1[012])[\/\-]\d{4}$'
    cnpj_pattern = '[0-9]{2}\.?[0-9]{3}\.?[0-9]{3}\/?[0-9]{4}\-?[0-9]{2}'
    val_pattern = '[0-9]'

    cupom_fulltext = pytesseract.image_to_string(img_gray, config=custom_config)

    nome_estabelecimento = re.match('\A.*', cupom_fulltext).group()

    cnpj_search = re.search(cnpj_pattern, cupom_fulltext)
    if cnpj_search != None:
        cnpj = cnpj_search.group()
    data_search = re.search(date_pattern, cupom_fulltext)
    if data_search != None:
        data_compra = data_search.group()

    isTotal = False
    hasTotal = False

    cupom_data = pytesseract.image_to_data(img_gray, output_type=Output.DICT, lang='por',config=custom_config)

    n_boxes = len(cupom_data['text'])
    for t in range(n_boxes):
        texto = cupom_data['text'][t]
        if re.match(date_pattern, texto):
            data_compra = texto
        if re.match(cnpj_pattern, texto):
            cnpj = texto
        if re.match('total', texto, re.IGNORECASE) and hasTotal == False:
            isTotal = True
        if isTotal:
            if re.match(val_pattern, texto):
                total = texto
                isTotal = False
                hasTotal = True


    print(f'Nome: {nome_estabelecimento}')
    print(f'CNPJ: {cnpj}')
    print(f'Data: {data_compra}')
    print(f'Total: {total}')
    print('\n')

    return nome_estabelecimento, cnpj, data_compra, total


In [4]:
def iterar_arquivos():
    dirName = 'arquivos'
    file_list = os.listdir(dirName)
    for entry in file_list:
        arquivo = os.path.join(dirName, entry)
        dados = busca_dados(arquivo)
        if dados != None:
            salvar_ocr(arquivo, dados)

In [5]:
def salvar_ocr(arquivo, dados):
    nome_arquivo, extensao_arquivo = os.path.splitext(arquivo)
    with open(nome_arquivo + '.txt', 'w') as out:
        out.write(f'Estabalecimento: {dados[0]}\n')
        out.write(f'CNPJ: {dados[1]}\n')
        out.write(f'Data: {dados[2]}\n')
        out.write(f'Total: {dados[3]}')

# Main

In [6]:
iterar_arquivos()

Extensão: .jpg
Processando: arquivos\cupom2
Nome: LOJAS RENNER S.A.
CNPJ: 92.754.738/0013-04
Data: 03/05/2013
Total: 288,90


Extensão: .txt
Extensão: .jpg
Processando: arquivos\cupom3
Nome: CUPOM FISCAL
CNPJ: 
Data: 05/02/2015
Total: 200,00


Extensão: .txt
Extensão: .jpg
Processando: arquivos\cupom4
Nome: Epson do Brasil Ind, e Com.
CNPJ: 52.106.911/0001-00
Data: 07/07/2007
Total: 21.00


Extensão: .txt
